In [ ]:
import os
import pandas as pd
import numpy as np
import anndata
import time
import matplotlib.pyplot as plt
import json
import requests
import pickle
import gzip as gz

import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src/')
from CellType_PSY import *

#### Functions

In [ ]:
def SaveDict(Dict, fname):
    with open(fname, 'wb') as hand:
        pickle.dump(Dict, hand)
    return

def LoadDict(fname):
    with open(fname, 'rb') as hand:
        b = pickle.load(hand)
        return b

# Seperate V2 and V3 data

In [ ]:
cell_extended = pd.read_csv("dat/cell_metadata_with_cluster_annotation.csv")

In [ ]:
cell_extended.columns.values

In [ ]:
cell_extended["library_label"]

In [ ]:
#label="subclass"
library_label = cell_extended["dataset_label"].unique()
library_label.sort()
library_label

In [ ]:
subclass = "046 Vip Gaba"
tmp = cell_extended[cell_extended["subclass"]==subclass]
tmp["feature_matrix_label"].value_counts()

In [ ]:
subclass = "256 SPVC Mafa Glut"
tmp = cell_extended[cell_extended["subclass"]==subclass]
tmp["feature_matrix_label"].value_counts()

In [ ]:
subclass = "280 NLL-po Pax7 Gaba"
tmp = cell_extended[cell_extended["subclass"]==subclass]
tmp["feature_matrix_label"].value_counts()

In [ ]:
subclass = "254 VCO Mafa Meis2 Glut"
tmp = cell_extended[cell_extended["subclass"]==subclass]
tmp["feature_matrix_label"].value_counts()

In [ ]:
subclass = "039 OB Meis2 Thsd7b Gaba"
tmp = cell_extended[cell_extended["subclass"]==subclass]
tmp["feature_matrix_label"].value_counts()

In [ ]:

###  Test similarity of same CT from V2 and V3

In [ ]:
DIR = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/AggregatedExpMat/"
Exp_v2_Isocortex_1 = pd.read_csv(DIR + "ExpMat.WMB-10Xv2-Isocortex-1.subclass.csv", index_col=0)
Exp_v2_Isocortex_2 = pd.read_csv(DIR + "ExpMat.WMB-10Xv2-Isocortex-2.subclass.csv", index_col=0)
Exp_v2_Isocortex_3 = pd.read_csv(DIR + "ExpMat.WMB-10Xv2-Isocortex-3.subclass.csv", index_col=0)
Exp_v2_Isocortex_4 = pd.read_csv(DIR + "ExpMat.WMB-10Xv2-Isocortex-4.subclass.csv", index_col=0)
Exp_v3_Isocortex_1 = pd.read_csv(DIR + "ExpMat.WMB-10Xv3-Isocortex-1.subclass.csv", index_col=0)
Exp_v3_Isocortex_2 = pd.read_csv(DIR + "ExpMat.WMB-10Xv3-Isocortex-2.subclass.csv", index_col=0)

In [ ]:
#CT = "046 Vip Gaba"
#CT = "053 Sst Gaba"
CT = "052 Pvalb Gaba"
#CT = "021 L4 RSP-ACA Glut"
#CT = "004 L6 IT CTX Glut"
#CT = "005 L5 IT CTX Glut"

In [ ]:
dat_v2 = np.concatenate([Exp_v2_Isocortex_1[CT].values, Exp_v2_Isocortex_2[CT].values,
                        Exp_v2_Isocortex_3[CT].values, Exp_v2_Isocortex_4[CT].values])
dat_v3 = np.concatenate([Exp_v3_Isocortex_1[CT].values, Exp_v3_Isocortex_2[CT].values])
sns.kdeplot(dat_v2, label="V2")
sns.kdeplot(dat_v3, label="V3")
plt.legend()

In [ ]:
fig, ax = plt.subplots(2,3, dpi=120)
ax[0,0].scatter(Exp_v2_Isocortex_1[CT].values, Exp_v2_Isocortex_2[CT].values, s=0.1)
ax[0,0].set_xlabel("v2_Isocortex_1"); ax[0,0].set_ylabel("v2_Isocortex_2")
ax[0,1].scatter(Exp_v2_Isocortex_1[CT].values, Exp_v2_Isocortex_3[CT].values, s=0.1)
ax[0,1].set_xlabel("v2_Isocortex_1"); ax[0,1].set_ylabel("v2_Isocortex_3")
ax[0,2].scatter(Exp_v2_Isocortex_1[CT].values, Exp_v2_Isocortex_4[CT].values, s=0.1)
ax[0,2].set_xlabel("v2_Isocortex_1"); ax[0,2].set_ylabel("v2_Isocortex_4")
ax[1,0].scatter(Exp_v2_Isocortex_1[CT].values, Exp_v3_Isocortex_1[CT].values, s=0.1)
ax[1,0].set_xlabel("v2_Isocortex_1"); ax[1,0].set_ylabel("v3_Isocortex_1")
ax[1,1].scatter(Exp_v2_Isocortex_1[CT].values, Exp_v3_Isocortex_2[CT].values, s=0.1)
ax[1,1].set_xlabel("v2_Isocortex_1"); ax[1,1].set_ylabel("v3_Isocortex_2")
ax[1,2].scatter(Exp_v3_Isocortex_1[CT].values, Exp_v3_Isocortex_2[CT].values, s=0.1)
ax[1,2].set_xlabel("v3_Isocortex_1"); ax[1,2].set_ylabel("v3_Isocortex_2")
fig.suptitle(CT, fontsize=12)
plt.tight_layout()



In [ ]:
Exp_v2_HPF = pd.read_csv(DIR + "ExpMat.WMB-10Xv2-HPF.subclass.csv", index_col=0)
Exp_v3_HPF = pd.read_csv(DIR + "ExpMat.WMB-10Xv3-HPF.subclass.csv", index_col=0)

In [ ]:
CTs = ["046 Vip Gaba", "053 Sst Gaba", "052 Pvalb Gaba",
       "036 HPF CR Glut", "037 DG Glut", "030 L6 CT CTX Glut"]
fig, axs = plt.subplots(2,3, dpi=120)
for nn, ax in enumerate(axs.flat):
    ax.scatter(Exp_v2_HPF[CTs[nn]].values, Exp_v3_HPF[CTs[nn]].values, s=0.1)
    ax.set_xlabel("v2_HPF"); ax.set_ylabel("v3_HPF")
    ax.set_title(CTs[nn])
#fig.suptitle(CT, fontsize=12)
plt.tight_layout()

## Check Cell Number

In [ ]:
ClusterMeta = pd.read_excel("/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx", 
                            index_col="cluster_id", sheet_name="cluster_annotation")

In [ ]:
ClusterMeta.head(2)

In [ ]:
ClusterMeta["TotalCell"] = ClusterMeta["v3.size"] + ClusterMeta["v2.size"]

In [ ]:
print(ClusterMeta[ClusterMeta["TotalCell"]>10].shape[0])
print(ClusterMeta[ClusterMeta["TotalCell"]>100].shape[0])
print(ClusterMeta[ClusterMeta["TotalCell"]>1000].shape[0])

In [ ]:
ClusterMeta[ClusterMeta["TotalCell"]>10].groupby("neighborhood")

In [ ]:
count_result = ClusterMeta[ClusterMeta["TotalCell"]>10]['neighborhood'].value_counts()
count_result

In [ ]:
count_result = ClusterMeta[ClusterMeta["TotalCell"]>100]['neighborhood'].value_counts()
count_result

In [ ]:
count_result = ClusterMeta[ClusterMeta["TotalCell"]>500]['neighborhood'].value_counts()
count_result